In [1]:
import numpy as np
import scipy.optimize as optimize
import scipy, math
import matplotlib.pyplot as plt
from functools import reduce 

# Construct the supersystem hamiltonian
def construct_h_super(aux_v, aux_e):
    h = np.zeros((n_super,n_super))
    h[:nphys,:nphys] = h_phys
    h[nphys:,nphys:] = np.diag(aux_e)
    h[:nphys,nphys:] = aux_v
    h[nphys:,:nphys] = aux_v.T
    assert(np.allclose(h,h.T))
    #print('aux_e', aux_e)
    #print('aux_v', aux_v)

    return h

# Construct the X-Matrix
def construct_X(e, v, mu, nmom):
    d = np.diag(np.power(e - mu, nmom))
    X = 2 * reduce(np.dot, (v[:nphys,:], d, v[:nphys,:].T))
    return X
    
# Construct T, it should have some Physicality and should not be random
def construct_T(nmom):
    #if nmom == 0:
       #T = 2*np.eye(nphys)
    #else:
       #T = np.random.random((nphys,nphys))
       #T = np.loadtxt('central_moments.txt', dtype = 'complex128')
    T_L = np.loadtxt('T2')
    #T = [[T_L[mom]] for mom in range(n+1)]
    T =  [np.zeros((nphys, nphys)) for mom in range(n+1)]
    for mom in range(n+1):
      T[mom] = [[T_L[mom]]]
    return T

# Calculate the error depending on whether gradients are needed
def mom_err(aux_param, T, calc_grad=False, ph_symmetry = False):

   if ph_symmetry:
      ener = aux_param[:naux].copy()
      coup = np.reshape(aux_param[naux:].copy(), (nphys,naux))
      aux_e_opt = np.hstack((-ener, ener))
      aux_v_opt = np.hstack((coup, coup))
      #print('e', aux_e_opt)
      #print('v', aux_v_opt)
    
   else:
      aux_e_opt = aux_param[:naux].copy() 
      aux_v_opt = np.reshape(aux_param[naux:].copy(), (nphys,naux))

   h = construct_h_super(aux_v_opt, aux_e_opt)
   e, v = np.linalg.eigh(h)
   mu = calc_chempot(e)
   #print('e_mom', e)
   #print('h_mom', h)

   #Construct all the X matrices here
   X = [construct_X(e, v, mu, mom)  for mom in range(n+1)]
   #assert(0)

   # sum_{n}  w_n | MF_mom(n) - HF_mom(n) |^2
   err = sum([np.sum(((X[mom]-T[mom])**2))/(scipy.math.factorial(mom)**2)  for mom in range(n+1)]) 
    
   if calc_grad:
      der_X = calc_deriv(e,v,mu)
      grad = np.zeros((nparam))
      for var in range(nparam):
        grad[var] = 2.*sum([np.sum((X[mom]-T[mom])*der_X[mom][var])  for mom in range(n+1)])
      return err, grad
   else:
      return err

#Define the chemical potential
def calc_chempot(eigenvalues):
    chempot = np.sum(eigenvalues)/n_super
    return chempot

#Define a function for calculating the derivatives
def calc_deriv(evals, evecs, chempot):

    deriv_evals = []
    deriv_evecs = []
    deriv_chempot = []

    for var in range(nparam):
        ham_list = construct_list_ham(var)
        Y = np.asarray([np.sum([evecs[ind[0],i]*evecs[ind[1],i] for ind in ham_list]) for i in range(n_super)])
        deriv_evals.append(Y)

        deriv_chempot.append(calc_chempot(Y))

        Z = np.zeros((n_super,n_super))
        for i in range(n_super-1):
            for j in range(i+1,n_super):
                delta = evals[j] - evals[i]
                if abs(delta) > 1.e-9:
                  Z[i,j] = -np.sum([evecs[ind[0],j]*evecs[ind[1],i] for ind in ham_list]) / delta
                  Z[j,i] = -Z[i,j]

        deriv_evecs.append(np.dot(evecs,Z.T))

    deriv = [[construct_X_deriv(evals, evecs, chempot, deriv_evals[var], deriv_evecs[var], deriv_chempot[var], mom)  for var in range(nparam)]  for mom in range(n+1)]

    return deriv

#Define a function for defining the structure of the derivatives
def construct_X_deriv(e, v, mu, e_deriv, v_deriv, mu_deriv, nmom):
    X_deriv = 2*reduce(np.dot, (v_deriv[:nphys,:], np.diag(np.power(e - mu, nmom)), v[:nphys,:].T))
    X_deriv += X_deriv.T
    X_deriv += 2*nmom * reduce(np.dot, (v[:nphys,:], np.diag(np.power(e - mu, nmom-1)*(e_deriv - mu_deriv)), v[:nphys,:].T))
    return X_deriv

def construct_list_ham(var):

    element = []

    if var < naux:
       ind = var + nphys
       element.append((ind,ind))
    else:
       element.append(coupling_list[var-naux])

    return element

#Define a function for doing the fitting / minimisation
def fitting(opt_params, grads=False, ph_symmetry = False):

    maxiter = 1000
    if grads:
       res = optimize.minimize(mom_err, opt_params, args=(T,True, ph_symmetry),
                method='l-bfgs-b',jac=True,tol=1e-9,options={'disp':True,'ftol':1.e-9})
    else:
       res = optimize.minimize(mom_err, opt_params, args=(T, False, ph_symmetry),
           method='nelder-mead',tol=1e-9,options={'xtol':1e-8,'disp':True,'maxiter':maxiter,'maxfev':maxiter})

    opt_aux_e = res.x[:naux]
    opt_aux_v = np.reshape(res.x[naux:], (nphys, naux))

    return opt_aux_e, opt_aux_v

#Compute the Physical system eigenvalues and eigenvectors
#Define initial energies and couplings

n = 5
naux = 1
nphys = 1
ph_symmetry = True

#h_phys = np.random.random((nphys,nphys))    #h_alpha,beta
#h_phys += h_phys.T
h_phys = np.zeros((nphys, nphys))
aux_e_init = np.array([[2.144483511249364316e+00]])[0]
aux_v_init = np.array([[2.792497045783768339e+00]])

if ph_symmetry:
    n_super = nphys + 2* naux
    ener = aux_e_init.copy()
    coup = np.reshape(aux_v_init.copy(), (nphys,naux))
    aux_e_symm = np.hstack((-ener, ener))
    aux_v_symm = np.hstack((coup, coup))
else:
    n_super = nphys + naux

nparam = naux + naux*nphys
print(aux_v_init)
print(aux_e_init)

coupling_list = []
for i in range(nphys):
  for j in range(nphys,n_super):
    coupling_list.append((i,j))

# Initial Energies and couplings here, with initial chemical potential and eigvals, eigvecs
if ph_symmetry:
    h_super_init = construct_h_super(aux_v_symm, aux_e_symm)  
else:
    h_super_init = construct_h_super(aux_v_init, aux_e_init)
    
#print('h_super', h_super_init)
e_super_init, v_super_init = np.linalg.eigh(h_super_init)
mu_super_init = calc_chempot(e_super_init)
mu_super_init = 0.
#print('e_super', e_super_init)

# Construct the initial X matrix here
X_init = [construct_X(e_super_init, v_super_init, mu_super_init, mom)  for mom in range(n+1)]

# Construct the T matrix here
T = construct_T(n)  
print('T_new', T)

param_init = np.zeros((nparam)) 
param_init[:naux] = aux_e_init
param_init[naux:] = np.reshape(aux_v_init, (nphys*naux))

# Initial error
error_init = mom_err(param_init, T, ph_symmetry = True)

# Final Energies and couplings here, with final chemical potential and eigvals, eigvecs
aux_e_final, aux_v_final = fitting(param_init, grads=False, ph_symmetry = True)

#print('ef', aux_e_final)
#print('vf', aux_v_final)

if ph_symmetry:
    if naux == 1:
        ener = aux_e_final[0].copy()
    else:   
        ener = aux_e_final.copy()
    coup = np.reshape(aux_v_final.copy(), (nphys,naux))
    aux_e_symm = np.hstack((-ener, ener))
    aux_v_symm = np.hstack((coup, coup))
    #print('aef', aux_e_final)
    h_super_final = construct_h_super(aux_v_symm, aux_e_symm)
else:
    h_super_final = construct_h_super(aux_v_final, aux_e_final)
    
e_super_final, v_super_final = np.linalg.eigh(h_super_final)
mu_super_final = calc_chempot(e_super_final)

# Construct the final X matrix here
X_final = [construct_X(e_super_final, v_super_final, mu_super_final, mom)  for mom in range(n+1)]
#print('esf', e_super_final)

new_param = np.zeros((nparam)) 
new_param[:naux] = aux_e_final
new_param[naux:] = np.reshape(aux_v_final, (nphys*naux))

# Final error
error_fin = mom_err(new_param, T, ph_symmetry = True)

print('HL mom', T)
print('Init param', param_init)
print('Init X', X_init)
print('Init error', error_init)
print()
print('Final param', new_param)
print('Final X', X_final)
print('Final error', error_fin)


[[2.79249705]]
[2.14448351]
T_new [[[2.0]], [[3.774758283725532e-15]], [[31.1921590028443]], [[2.1316282072803006e-14]], [[629.9221897135524]], [[-5.4569682106375694e-12]]]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 59
         Function evaluations: 119
HL mom [[[2.0]], [[3.774758283725532e-15]], [[31.1921590028443]], [[2.1316282072803006e-14]], [[629.9221897135524]], [[-5.4569682106375694e-12]]]
Init param [2.14448351 2.79249705]
Init X [array([[2.]]), array([[-3.58927059e-15]]), array([[31.192159]]), array([[-1.56319402e-13]]), array([[629.92218971]]), array([[-4.09272616e-12]])]
Init error 3.5080087211100484e-27

Final param [2.14448351 2.79249705]
Final X [array([[2.]]), array([[-3.72410878e-15]]), array([[31.192159]]), array([[-1.56319402e-13]]), array([[629.92218971]]), array([[-4.09272616e-12]])]
Final error 3.5080087211100484e-27
